In [1]:
import xml.etree.ElementTree as ET
import os

In [22]:

# Function to parse the XML file
def parse_node_xml(file_path):
# Parse the XML file
    tree = ET.parse(file_path)
    root = tree.getroot()

    file_name = os.path.splitext(os.path.basename(file_path))[0]
    # Dictionary to hold parsed content
    parsed_data = {'Name':file_name}

    # Iterate over language nodes
    for language in root.findall('language'):
        lang_code = language.get('value')
        parsed_data[lang_code] = {
            'descriptions': {},
            'nodePhrases': {}
        }

        # Extract descriptions by level (Novice, Intermediate, Expert)
        for description in language.findall('description'):
            level = description.get('level')
            text = description.text.strip() if description.text else "No description available"
            parsed_data[lang_code]['descriptions'][level] = text

        # Extract node phrases by level (Novice, Intermediate, Expert)
        for nodePhrase in language.findall('nodePhrase'):
            level = nodePhrase.get('level')
            clipId = nodePhrase.get('clipId')
            text = nodePhrase.text.strip() if nodePhrase.text else "No node phrase available"
            
            if level not in parsed_data[lang_code]['nodePhrases']:
                parsed_data[lang_code]['nodePhrases'][level] = {}

            parsed_data[lang_code]['nodePhrases'][level][clipId] = text

    return parsed_data


def printParsedXml(parsed_content):
    for language, content in parsed_content.items():
        if(language=="Name"):
            print(f"{content}\n")
            continue

        print(f"Language: {language}")

        print("Descriptions:")
        for level, description in content['descriptions'].items():
            print(f"  {level}: {description}")
    
        print("\nNode Phrases:")
        for level, phrases in content['nodePhrases'].items():
            print(f"  {level}:")
            for clipId, phrase in phrases.items():
                print(f"    {clipId}: {phrase}")
    
        print("\n")

In [23]:
# Path to the XML file
file_path = r"..\public\xmls\Educational-Contents\BLSDPro\DefibrillatorInit.xml"

# Call the function and print parsed data
parsed_content = parse_node_xml(file_path)
# Output the parsed descriptions and node phrases
printParsedXml(parsed_content)

DefibrillatorInit

Language: ITA
Descriptions:
  Novice: Posiziona le placche adesive come indicato sul torace del paziente e collegale al defibrillatore tramite lo spinotto.
  Intermediate: Applica le placche e attacca lo spinotto.
  Expert: Applica le placche e attacca lo spinotto.

Node Phrases:
  Novice:
    DefibrillatorInitPaddlePlaced: piastra in posizione
    DefibrillatorInitPaddlesPlaced: Tutte le piastre sono correttamente posizionate.
    DefibrillatorInitPlugConnected: spinotto inserito
    DefibrillatorInitUserInfoPaddles: posizionato le piastre sul petto
    DefibrillatorInitUserInfoPlug: inserito lo spinotto
    DefibrillatorInitUserInfoStart: Non hai ancora


Language: ENG
Descriptions:
  Novice: Place the adhesive pads as indicated on the patient's chest and connect them to the defibrillator via the plug.
  Intermediate: Attach the pads and plug in the connector.
  Expert: Attach the pads and plug in the connector.

Node Phrases:
  Novice:
    DefibrillatorInitPaddleP

In [18]:
x=parsed_content.items()

for key,value in x:
    print(f"key: {key}    value: {value}\n")

key: Name    value: DefibrillatorInit.xml

key: ITA    value: {'descriptions': {'Novice': 'Posiziona le placche adesive come indicato sul torace del paziente e collegale al defibrillatore tramite lo spinotto.', 'Intermediate': 'Applica le placche e attacca lo spinotto.', 'Expert': 'Applica le placche e attacca lo spinotto.'}, 'nodePhrases': {'Novice': {'DefibrillatorInitPaddlePlaced': 'piastra in posizione', 'DefibrillatorInitPaddlesPlaced': 'Tutte le piastre sono correttamente posizionate.', 'DefibrillatorInitPlugConnected': 'spinotto inserito', 'DefibrillatorInitUserInfoPaddles': 'posizionato le piastre sul petto', 'DefibrillatorInitUserInfoPlug': 'inserito lo spinotto', 'DefibrillatorInitUserInfoStart': 'Non hai ancora'}}}

key: ENG    value: {'descriptions': {'Novice': "Place the adhesive pads as indicated on the patient's chest and connect them to the defibrillator via the plug.", 'Intermediate': 'Attach the pads and plug in the connector.', 'Expert': 'Attach the pads and plug in 

In [15]:
print(parsed_content)

{'Name': 'DefibrillatorInit.xml', 'ITA': {'descriptions': {'Novice': 'Posiziona le placche adesive come indicato sul torace del paziente e collegale al defibrillatore tramite lo spinotto.', 'Intermediate': 'Applica le placche e attacca lo spinotto.', 'Expert': 'Applica le placche e attacca lo spinotto.'}, 'nodePhrases': {'Novice': {'DefibrillatorInitPaddlePlaced': 'piastra in posizione', 'DefibrillatorInitPaddlesPlaced': 'Tutte le piastre sono correttamente posizionate.', 'DefibrillatorInitPlugConnected': 'spinotto inserito', 'DefibrillatorInitUserInfoPaddles': 'posizionato le piastre sul petto', 'DefibrillatorInitUserInfoPlug': 'inserito lo spinotto', 'DefibrillatorInitUserInfoStart': 'Non hai ancora'}}}, 'ENG': {'descriptions': {'Novice': "Place the adhesive pads as indicated on the patient's chest and connect them to the defibrillator via the plug.", 'Intermediate': 'Attach the pads and plug in the connector.', 'Expert': 'Attach the pads and plug in the connector.'}, 'nodePhrases': 

## Test parsing .procedure file

In [3]:
import os
import json
from xml.etree.ElementTree import Element, SubElement, tostring
from xml.dom.minidom import parseString
from io import BytesIO
import zipfile

In [10]:
# Funzione per creare l'XML
def create_activity_xml(activity):
    root = Element('node')
    # Lingua italiana
    language_xml(root,activity.ita,'ITA')
    language_xml(root,activity.eng,'ENG')

    # Convertire l'albero XML in una stringa formattata
    xml_str = parseString(tostring(root, encoding="unicode")).toprettyxml(indent="   ").replace("&quot;","\"")

    return xml_str

def language_xml(root,activityLang,lang):
    # Lingua italiana
    ita = SubElement(root, 'language', value=lang)
    for phrase in activityLang.node_phrases:
        if phrase['clipId'] == 'Description':
            desc = SubElement(ita, 'description', level=phrase['level'])
            desc.text = phrase.get('text', phrase['level'])
        else:
            node_phrase = SubElement(ita, 'nodePhrase', clipId=phrase['clipId'], level=phrase['level'])
            node_phrase.text = phrase.get('text', phrase['clipId'])
    details=SubElement(ita,'details',level='Novice')
    details.text=activityLang.details

def retrieveAllActivitiesFromProcedure(data):
    activities = []

    for node in data.get('flow', {}).get('nodes', []):
        #retrieve the single activity
        activity = node.get('data', {}).get('activity', None)
        if not activity: 
            continue
        
        #activity contains aslo subProcedure but we are interested only in the name e phrases
        slimActivity= Activity(activity.get('name',""), activity.get('languages',{}))
        activities.append(slimActivity)

        #Retrieve all the activities present in the subProcedure
        # subProcedure=activity.get('subProcedure',None)
        # if not subProcedure:
        #     continue

        # subProcedureActivities=retrieveAllActivities(subProcedure)
        # if not subProcedureActivities:
        #     continue
        # #append each sub-activities
        # for subProcedureActivity in subProcedureActivities:
        #     activities.append(subProcedureActivity)

    return activities

def retrieveAllActivities(data):
    activities=[]
    for item in data:
        if "flow" in item and "nodes" in item["flow"]:
            activities.extend(retrieveAllActivitiesFromProcedure(item))
    return activities

class Activity:
    def __init__(self, name, languages):
        self.name = name  
        itaLang=languages.get('itaActivity',None)
        itaPhrase=itaLang.get('nodePhrases',[])
        itaDetails=itaLang.get('details',"")
        self.ita = LanguageActivity(itaPhrase,itaDetails)
        engLang=languages.get('engActivity',None)
        engPhrase=engLang.get('nodePhrases',[])
        engDetails=engLang.get('details',"")
        self.eng = LanguageActivity(engPhrase,engDetails)
    
    def printSinglePhrase(self,phrase):
        return f"{phrase['clipId']} {phrase['level']}: {phrase['text']}"

    def __str__(self):
        phrases=""
        for phrase in self.node_phrases:
            phrases +="  "+ self.printSinglePhrase(phrase)+"\n"

        return f"Start\n {self.name}\n NodePhrases:\n{phrases}END"  # Correct: Returning a string

class LanguageActivity:
    def __init__(self,node_phrases,details):
        self.details = details  
        self.node_phrases = node_phrases


def zipAllActivitiesXmls(data):
    activities=retrieveAllActivities(data)
    xmls=[]
    for activity in activities:
        xml_content=create_activity_xml(activity)
        #writeXMLFile(output_dir,xml_content,activity.name)
        xmls.append((f"{activity.name}.xml", xml_content))
    # Preparare un archivio ZIP contenente tutti i file XML
        print(xml_content)
    zip_buffer = BytesIO()

    with zipfile.ZipFile(zip_buffer, 'w', zipfile.ZIP_DEFLATED) as zip_file:
        zip_file.writestr(f"{data[0].get('title','no-title')}.procedure", json.dumps(data, ensure_ascii=False))
        for filename, xml_content  in xmls:
            zip_file.writestr(filename, xml_content )
    zip_buffer.seek(0)
    return zip_buffer

In [11]:
# Percorso del file
input_file=r"D:\Downloads\TEST(2).procedure"
output_dir =r"D:\Downloads\generatedXML"

def extract_XML_from_procedure(data):
    zip_folder=zipAllActivitiesXmls(data)

with open(input_file, 'r', encoding='utf-8') as file:
    data = json.load(file)
    extract_XML_from_procedure(data)

<?xml version="1.0" ?>
<node>
   <language value="ITA">
      <description level="Expert">Posiziona la maschera.</description>
      <description level="Intermediate">Posiziona la maschera.</description>
      <description level="Novice">Sul pavimento troverai una maschera, afferrala e posizionala correttamente sulla bocca del paziente.</description>
      <nodePhrase clipId="ApplyBreathMaskUserInfoMaskNotPositioned" level="Novice">Hai afferrato la maschera ma non l'hai posizionata nel punto giusto. Sulla bocca del paziente trovi un riferimento.</nodePhrase>
      <nodePhrase clipId="ApplyBreathMaskUserInfoNoMaskGrabbed" level="Novice">Non hai ancora afferrato la maschera. Usa una mano per afferrarla e posizionala sulla bocca del paziente.</nodePhrase>
      <details level="Novice"/>
   </language>
   <language value="ENG">
      <description level="Expert">Position the mask.</description>
      <description level="Intermediate">Position the mask.</description>
      <description level

In [14]:
# for activity in activities:
#     xml_content = create_activity_xml(activity.node_phrases)
#     print(xml_content)
activity=activities[0]
xml_content = {activity.name,create_activity_xml(activity.node_phrases)}
print(xml_content)
# writeXMLFile(output_dir,xml_content,activity.name)

{'AskDefibrillator', '<?xml version="1.0" ?>\n<node>\n   <language value="ITA">\n      <description level="Expert">Prendi un defibrillatore. Se non sei solo, chiedi a qualcuno di procurarti un defibrillatore. Non muoverti dalla scena. Unico passaggio con "call emergency"</description>\n      <description level="Intermediate">Prendi un defibrillatore. Se non sei solo, chiedi a qualcuno di procurarti un defibrillatore. Non muoverti dalla scena. Unico passaggio con "call emergency"</description>\n      <description level="Novice">Prendi un defibrillatore. Se non sei solo, chiedi a qualcuno di procurarti un defibrillatore. Non muoverti dalla scena.</description>\n      <nodePhrase clipId="AskDefibrillatorUserInfo1" level="Novice">Guarda intorno a te. C\'è qualcuno nella stanza che può portarti il defibrillatore</nodePhrase>\n      <nodePhrase clipId="AskDefibrillatorUserInfo2" level="Novice">Non hai ancora chiesto un defibrillatore. Guarda meglio intorno a te. C\'è qualcuno nella stanza ch

In [16]:
xmls=[xml_content]
# Preparare un archivio ZIP contenente tutti i file XML
zip_buffer = BytesIO()
with zipfile.ZipFile(zip_buffer, 'w') as zip_file:
    for filename, xml_file in xmls:
        zip_file.writestr(filename, xml_file)
zip_buffer.seek(0)

0